In [1]:
from handler_collision_with_wall import handler_wall_collision as handler_wall_collision_cy
from data_structures.vector import MyVector
import numpy as np

In [2]:
def handler_wall_collision(position, speed, radius, wall_directing_vector, wall):
    # p index of the part
    x1, y1, x2, y2 = wall # x1<=x2 etc.

    # angle
    theta = np.sign(wall_directing_vector.y)*np.arccos(wall_directing_vector.x)

    # A and B
    sTheta, cTheta = np.sin(theta), np.cos(theta)
    B = -speed.x*sTheta+speed.y*cTheta
    if B == 0.0 : 
        return np.nan, None # TODO : should we add a tolerance ? It will never be equals to zero exactly...
    A = -position.x*sTheta+position.y*cTheta

    # new position of the wall in the new base
    y1_new_base = -x1*sTheta+y1*cTheta
    A_prime = A-y1_new_base
    # possible collision time :
    t_coll_1 = (-A_prime-radius)/B
    t_coll_2 = (-A_prime+radius)/B

    t_intersect = max(t_coll_1, t_coll_2)

    if(t_intersect > 0):
        # t_intersect = max(t_coll_1, t_coll_2) # because we are not anticipating them anymore
        # t_intersect = -A_prime/B # the time at which the disk crosses the line if its radius were radius=0.
        pos_intersect = position + t_intersect * speed

        wall_extrimity1_coordinates = MyVector(x1,y1)
        wall_extrimity2_coordinates = MyVector(x2,y2)
        # the reason why were are not using pos_intersect.norm is that it should be a 3D vector.
        dP1, dP2, dP3 = wall_extrimity2_coordinates-wall_extrimity1_coordinates, \
            pos_intersect-wall_extrimity1_coordinates, wall_extrimity2_coordinates-pos_intersect 
        norm_1 = dP1.norm()
        norm_2 = dP2.norm()
        # norm_3 = dP3.norm()
        qty=dP1.inner(dP2)/(norm_1*norm_1) # norm_1 cant be 0 because wall segments are not on same points.
        if(qty < 1 and qty > 0):
            return t_intersect, pos_intersect
    return np.nan, None

In [3]:
def wrapper(position, speed, radius, wall_directing_vector, wall):
    return handler_wall_collision_cy(position.x, position.y, speed.x, speed.y, radius, wall[0], wall[1], wall[2], wall[3], wall_directing_vector.x, wall_directing_vector.y)

In [4]:
%timeit handler_wall_collision(MyVector(0,0), MyVector(1,0), 0.1, MyVector(0,1), [1,-1, 1, 1]) 

30.2 µs ± 350 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
%timeit wrapper(MyVector(0,0), MyVector(1,0), 0.1, MyVector(0,1), [1.,-1., 1., 1.]) 

1.84 µs ± 13.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [6]:
%timeit handler_wall_collision_cy(0.,0., 1., 0., 0.1, 1., -1., 1., 1., 1., 0.)

137 ns ± 1.13 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [42]:
t_naive = 30200
t_wrapper = 1840
t_pure_c = 137
print(f'Pure C is {round(t_naive/t_pure_c,2)} quicker than the naive one. However, comparing with the wrapper, we have only a {round(t_naive/t_wrapper,2)} decrease in time.')

Pure C is 220.44 quicker than the naive one. However, comparing with the wrapper, we have only a 16.41 decrease in time.


In [10]:
print(handler_wall_collision(MyVector(0,0), MyVector(1,0), 0.1, MyVector(0,1), [1,-1, 1, 1]))
print(handler_wall_collision_cy(0.,0., 1., 0., 0.1, 1., -1., 1., 1., 0., 1.))
print(wrapper(MyVector(0,0), MyVector(1,0), 0.1, MyVector(0.,1.), [1.,-1., 1., 1.]))

(1.1, (1.1, 0.0))
(1.1, 1.1, 0.0)
(1.1, 1.1, 0.0)


In [16]:
from time import time
N = 100000
t_wrapper = 0
for k in range(N):
    t1 = time()
    wrapper(MyVector(0,0), MyVector(1,0), 0.1, MyVector(0.,1.), [1.,-1., 1., 1.])
    t2 = time()
    t_wrapper += t2-t1

In [17]:
N = 100000
t_naive = 0
for k in range(N):
    t1 = time()
    handler_wall_collision(MyVector(0,0), MyVector(1,0), 0.1, MyVector(0,1), [1,-1, 1, 1])
    t2 = time()
    t_naive += t2-t1

In [36]:
from time import time
N = 100000
t_pure_C=0
for k in range(N):
    t1 = time()
    handler_wall_collision_cy(0.,0., 1., 0., 0.1, 1., -1., 1., 1., 0., 1.)
    t2 = time()
    t_pure_C += t2-t1

In [37]:
print(f'Pure C is {t_naive/t_pure_C} quicker than the naive one. However, comparing with the wrapper, we have only a {t_naive/t_wrapper} decrease in time.')

Pure C is 108.75765414741916 quicker than the naive one. However, comparing with the wrapper, we have only a 15.924406875759454 decrease in time.
